<a id="1.1"></a>
<h3 style="background-color:orange;font-family:newtimeroman;font-size:300%;text-align:center;border-radius: 15px 50px;">Introduction</h3>


<p style="text-align: center;"><span style='font-family: "Times New Roman", Times, serif; font-size: 24px;'>Greetings my fellow Kagglers,&nbsp;</span></p>
<p style="text-align: center;"><span style="font-size: 24px;"><span style="font-family: 'Times New Roman', Times, serif;">While going through the enormous amount of notebooks with the solution to the digit recognition problem I got a little disappointed as it seemed that most of the notebooks revolve around CNN&apos;s and other ANN architectures.</span></span></p>
<p style="text-align: center;"><span style="font-size: 24px;"><span style="font-family: 'Times New Roman', Times, serif;"><br></span></span></p>
<p style="text-align: center;"><span style="font-size: 24px;"><span style="font-family: 'Times New Roman', Times, serif;">Without a doubt, some of the best if not all of the good results are the product of those &quot;typical&quot; solutions.</span></span></p>
<p style="text-align: center;"><span style="font-size: 24px;"><span style="font-family: 'Times New Roman', Times, serif;"><br></span></span></p>
<p style="text-align: center;"><span style='font-family: "Times New Roman", Times, serif; font-size: 24px;'>I decided to propose a different approach even though it is far-far away in terms of accuracy from CNN&apos;s but at the same time I believe that this amazing community is all about learning from each other and having so many notebooks which seem like copy-paste versions of one notebook is less insightful I believe.</span></p>
<p><br></p>

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.express as ex
import matplotlib.pyplot as plt
#%pip install opencv-contrib-python
import cv2 as cv
import seaborn as sns


plt.rc('figure',figsize=(17,11))


In [ ]:
train = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
test =  pd.read_csv('/kaggle/input/digit-recognizer/test.csv')
target = pd.get_dummies(train.label)
label = pd.DataFrame({'labels':train.pop('label')})
train[train<210]=0
train[train>0]=255
test[test<210]=0
test[test>0]=255

train.head(3)

In [ ]:
train_images = [img.values.reshape(28,28).astype(np.float64) for _,img in train.iterrows()]
test_images = [img.values.reshape(28,28).astype(np.float64) for _,img in test.iterrows()]

<a id="1.1"></a>
<h3 style="background-color:orange;font-family:newtimeroman;font-size:200%;text-align:center;border-radius: 15px 50px;">Probabilistic Infrence</h3>


In [ ]:
N = 784

digit_pixel_prob=[]
for i in range(0,10):
    ones = train.loc[label.query(f'labels=={i}').index,:]
    ones = ones/255
    pf = (ones.sum()/len(ones)).values
    ipf = pf.reshape((28,28))
    digit_pixel_prob.append(ipf)

fig, axes = plt.subplots(nrows=2, ncols=5)
number = 0
for ax in axes.flat:
    ax.set_title(f' Pixels Probabilites Num: [{number}]')
    im = ax.imshow(digit_pixel_prob[number],cmap='coolwarm',vmax=1)
    number+=1
fig.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([0.85, 0.12, 0.05, 0.7])
fig.colorbar(im, cax=cbar_ax)
plt.show()


<a id="1.1"></a>
<h3 style="background-color:orange;font-family:newtimeroman;font-size:300%;text-align:center;border-radius: 15px 50px;">Harris Corner Detection</h3>


<p style="text-align: center;"><span style='font-size: 24px; font-family: "Times New Roman", Times, serif;'>We will use the Harris Corner Detection algorithms to extract the corners (&quot;the significant&quot;) pixel&apos;s which will allow us later to construct a characteristic matrix for each digit.</span></p>
<p style="text-align: center;"><span style="font-family: 'Times New Roman', Times, serif;"><span style="font-size: 24px;">The logic of this particular method will be to find all the corners for each digit and construct a characteristic matrix.</span></span></p>
<p style="text-align: center;"><span style="font-family: 'Times New Roman', Times, serif;"><span style="font-size: 24px;">When we will be presented with a new digit we will find its corners and check which characteristic matrix describes best those corners.</span></span></p>
<p style="text-align: center;"><span style="font-family: 'Times New Roman', Times, serif;"><span style="font-size: 24px;"><br></span></span></p>
<p style="text-align: center;"><span style="font-family: 'Times New Roman', Times, serif;"><span style="font-size: 24px;">But before we dive into all the calculations let&apos;s first understand how will we calculate those Harris Corners.</span></span></p>
<p style="text-align: center;"><span style='font-size: 24px; font-family: "Times New Roman", Times, serif;'><br></span></p>
<p><br></p>

<p style="text-align: center;"><span style='font-size: 24px; font-family: "Times New Roman", Times, serif;'><strong><u>Harris Corners Detector Steps</u></strong></span></p>
<p style="text-align: center;"><span style="font-family: 'Times New Roman', Times, serif;"><span style="font-size: 24px;">1. Compute image gradients: Gx, Gy</span></span></p>
<p style="text-align: center;"><span style="font-family: 'Times New Roman', Times, serif;"><span style="font-size: 24px;">2. Compute products: Gx*Gx, Gx*Gy, Gy*Gy</span></span></p>
<p style="text-align: center;"><span style="font-family: 'Times New Roman', Times, serif;"><span style="font-size: 24px;">3. Filter products with a Gaussian window</span></span></p>
<p style="text-align: center;"><span style="font-family: 'Times New Roman', Times, serif;"><span style="font-size: 24px;">4. For each pixel (i,j) define the matrix M</span></span></p>
<p style="text-align: center;"><span style="font-family: 'Times New Roman', Times, serif;"><span style="font-size: 24px;">5. For each pixel compute the score R</span></span></p>
<p style="text-align: center;"><span style="font-family: 'Times New Roman', Times, serif;"><span style="font-size: 24px;">6. Threshold R, and perform non-maxima suppression</span></span></p>
<p style="text-align: center;"><span style='font-size: 24px; font-family: "Times New Roman", Times, serif;'><br></span></p>
<p><br></p>

<a id="1.1"></a>
<h3 style="background-color:orange;font-family:newtimeroman;font-size:200%;text-align:center;border-radius: 15px 50px;">1) Compute image gradients: Gx, Gy</h3>


In [ ]:
def calculate_image_gradients(img):
    #Derivative Kernels
    Kx = np.array([[-1,0,1],[-2,0,2],[-1,0,1]]) 
    Ky = Kx.T
    #Calculate The Derivative with respect to the x axis
    Gx = cv.filter2D(img,-1,Kx)
    #Calculate The Derivative with respect to the y axis
    Gy = cv.filter2D(img,-1,Ky)
    
    return Gx,Gy

In [ ]:
img = np.zeros((21,21), dtype=np.float32)
img[5:-5,5:-5]=200

Gx,Gy = calculate_image_gradients(img)

plt.subplot(2,3,1)
plt.title("Gx",fontsize=16,fontweight='bold')
plt.imshow(Gx,cmap='jet')
plt.subplot(2,3,2)
plt.title("Gy",fontsize=16,fontweight='bold')
plt.imshow(Gy,cmap='jet')
plt.subplot(2,3,3)
plt.title("Original Image",fontsize=16,fontweight='bold')
plt.imshow(img,cmap='jet')

#from our data
img = train_images[8].copy()
Gx,Gy = calculate_image_gradients(img)

plt.subplot(2,3,4)
plt.title("Gx",fontsize=16,fontweight='bold')
plt.imshow(Gx,cmap='jet')
plt.subplot(2,3,5)
plt.title("Gy",fontsize=16,fontweight='bold')
plt.imshow(Gy,cmap='jet')
plt.subplot(2,3,6)
plt.title("Original Image",fontsize=16,fontweight='bold')
plt.imshow(img,cmap='jet')

<a id="1.1"></a>
<h3 style="background-color:orange;font-family:newtimeroman;font-size:200%;text-align:center;border-radius: 15px 50px;">2) Compute the second order moments: ($Gx\cdot Gx$, $Gx\cdot Gy$, $Gy\cdot Gy$)</h3>


In [ ]:
titles=[r"$Gx^{2}$",r"$Gy^{2}$",r"$Gy\cdot Gx}$"]

img = np.zeros((21,21), dtype=np.float32)
img[5:-5,5:-5]=200
Gx,Gy = calculate_image_gradients(img)
moments = [Gx**2,Gx*Gy,Gy**2]

img = np.zeros((21,21), dtype=np.float32)
img[5:-5,5:-5]=200

for i in range(0,3):
    plt.subplot(2,3,i+1)
    plt.title(titles[i],fontsize=16,fontweight='bold')
    plt.imshow(moments[i],cmap='jet')

    
        
img = train_images[8].copy()
Gx,Gy = calculate_image_gradients(img)
moments = [Gx**2,Gx*Gy,Gy**2]

img = np.zeros((21,21), dtype=np.float32)
img[5:-5,5:-5]=200

for i in range(0,3):
    plt.subplot(2,3,3+(i+1))
    plt.title(titles[i],fontsize=16,fontweight='bold')
    plt.imshow(moments[i],cmap='jet')


<a id="1.1"></a>
<h3 style="background-color:orange;font-family:newtimeroman;font-size:200%;text-align:center;border-radius: 15px 50px;">3) Filter products with a Gaussian window</h3>


In [ ]:
titles=[r"$Gx^{2}$",r"$Gy^{2}$",r"$Gy\cdot Gx}$"]

img = np.zeros((21,21), dtype=np.float32)
img[5:-5,5:-5]=200
Gx,Gy = calculate_image_gradients(img)
moments = [Gx**2,Gx*Gy,Gy**2]
moments = [cv.GaussianBlur(moments[i],(3,3),2) for i in range(0,3)]

img = np.zeros((21,21), dtype=np.float32)
img[5:-5,5:-5]=200

for i in range(0,3):
    plt.subplot(2,3,i+1)
    plt.title('Filtered ' +titles[i],fontsize=16,fontweight='bold')
    plt.imshow(moments[i],cmap='jet')

    
        
img = train_images[8].copy()
Gx,Gy = calculate_image_gradients(img)
moments = [Gx**2,Gx*Gy,Gy**2]
moments = [cv.GaussianBlur(moments[i],(3,3),2) for i in range(0,3)]

img = np.zeros((21,21), dtype=np.float32)
img[5:-5,5:-5]=200

for i in range(0,3):
    plt.subplot(2,3,3+(i+1))
    plt.title('Filtered ' +titles[i],fontsize=16,fontweight='bold')
    plt.imshow(moments[i],cmap='jet')


<a id="1.1"></a>
<h3 style="background-color:orange;font-family:newtimeroman;font-size:200%;text-align:center;border-radius: 15px 50px;">4) For each pixel $(i,j)$ define the matrix $M$</h3>


We will define $M = \begin{bmatrix}
 m_{11}(i,j)&m_{12}(i,j) \\ 
 m_{21}(i,j)&m_{22}(i,j) 
\end{bmatrix}$ where each $m$ is on of the 4 moment matrix's calculated earlier

<a id="1.1"></a>
<h3 style="background-color:orange;font-family:newtimeroman;font-size:200%;text-align:center;border-radius: 15px 50px;">5) For each pixel compute the score $R$</h3>


For each (i,j) define a 2x2 moments-matrix:  $M = \begin{bmatrix}
 \sum{G_x^2}&\sum{G_xG_y} \\ 
 \sum{G_xG_y}&\sum{G_y^2} 
\end{bmatrix}$

We will define the Harris Score as : $R = det(M) - \alpha{[tr(M)]^2}$
Where det is the determinante operator and tr is the trace operator

In [ ]:
img = np.zeros((21,21), dtype=np.float32)
img[5:-5,5:-5]=200

plt.subplot(2,2,1)
plt.title('Original Image',weight='bold')
plt.imshow(img,cmap='jet')
plt.subplot(2,2,2)
Gx,Gy =calculate_image_gradients(img)
M = [Gx**2,Gx*Gy,Gy**2]
M = [cv.GaussianBlur(Prod,(3,3),2) for Prod in M]
detM = M[0]*M[2]-M[1]**2
traceM = M[0]+M[2]
R_scores = detM-0.06*(traceM**2)
plt.title('R Score Matrix',weight='bold')
plt.imshow(R_scores,cmap='jet')
plt.colorbar()

img = train_images[8].copy()

plt.subplot(2,2,3)
plt.title('Original Image',weight='bold')
plt.imshow(img,cmap='jet')
plt.subplot(2,2,4)
Gx,Gy =calculate_image_gradients(img)
M = [Gx**2,Gx*Gy,Gy**2]
M = [cv.GaussianBlur(Prod,(3,3),2) for Prod in M]
detM = M[0]*M[2]-M[1]**2
traceM = M[0]+M[2]
R_scores = detM-0.06*(traceM**2)
plt.title('R Score Matrix',weight='bold')
plt.imshow(R_scores,cmap='jet')
plt.colorbar()


<a id="1.1"></a>
<h3 style="background-color:orange;font-family:newtimeroman;font-size:200%;text-align:center;border-radius: 15px 50px;">6) Threshold $R$, and perform non-maxima suppression</h3>


In [ ]:
def get_R_scores(Gx,Gy,Sigma=2,alpha=0.06,tsh=0.35):
    M = [Gx**2,Gx*Gy,Gy**2]
    M = [cv.GaussianBlur(Prod,(3,3),Sigma) for Prod in M]
    detM = M[0]*M[2]-M[1]**2
    traceM = M[0]+M[2]
    R_scores = detM-alpha*(traceM**2)
    cross_kernel = np.ones((3,3),np.uint8)
    #cross_kernel[1,:]=1
    #cross_kernel[:,1]=1
    R_dilate = cv.dilate(R_scores,cross_kernel)
    R_th = R_scores > R_scores.max()*tsh; # threshold
    R_nms = R_scores >= R_dilate;# NMS
    R_final = R_th * R_nms # threshold and NMS
    #plt.imshow(R_scores,cmap='jet')
    return R_final.astype(np.int)


In [ ]:
img = np.zeros((21,21), dtype=np.float32)
img[5:-5,5:-5]=200

plt.subplot(2,2,1)
plt.title('Original Image',weight='bold')
plt.imshow(img,cmap='jet')
plt.subplot(2,2,2)
plt.title('R Score Matrix after NMS',weight='bold')
Gx,Gy =calculate_image_gradients(img)
plt.imshow(get_R_scores(Gx,Gy),cmap='jet')
plt.colorbar()

img = train_images[8].copy()

plt.subplot(2,2,3)
plt.title('Original Image',weight='bold')
plt.imshow(img,cmap='jet')
plt.subplot(2,2,4)
Gx,Gy =calculate_image_gradients(img)
plt.title('R Score Matrix',weight='bold')
plt.imshow(get_R_scores(Gx,Gy),cmap='jet')
plt.colorbar()


In [ ]:
def Calculate_Harris_Corners(img,Sigma=0.3,alpha=0.06,tsh=0.1):
    gx,gy = calculate_image_gradients(img)
    return get_R_scores(gx,gy,Sigma,alpha,tsh)


<p style="text-align: center;"><span style='font-family: "Times New Roman", Times, serif; font-size: 24px;'>The above is a utility function that unites all the steps we discussed below and will allow us to tweak the parameters if needed.<br></span></p>
<p><br></p>

<a id="1.1"></a>
<h3 style="background-color:orange;font-family:newtimeroman;font-size:200%;text-align:center;border-radius: 15px 50px;">Example Of Corners Detected For Our Train Images</h3>


In [ ]:
for i in range(0,6):
    plt.subplot(2,3,i+1)

    target = train_images[10+i]

    plt.imshow(target,cmap='gray',label='Harris Corners')
    y,x=np.nonzero(Calculate_Harris_Corners(target))
    plt.plot(x,y,'ro')
    plt.legend(["Corners"])


<a id="1.1"></a>
<h3 style="background-color:orange;font-family:newtimeroman;font-size:200%;text-align:center;border-radius: 15px 50px;">Calculating The Characteristic R Matrix For Each Digit</h3>


In [ ]:
HARRIS_BLOCK_SIZE = 3
HARRIS_K_SIZE     = 5
HARIS_ALPHA       = 0.03

characteristic_R_Matrix =[]

for number in range(0,10):
    aux =  np.zeros_like(train_images[0],np.float64)
    for zero_ind in label[label.labels==number].index:
        aux += cv.cornerHarris(np.float32(train_images[zero_ind]),HARRIS_BLOCK_SIZE,HARRIS_K_SIZE,HARIS_ALPHA)
    characteristic_R_Matrix.append((aux))


<a id="1.1"></a>
<h3 style="background-color:orange;font-family:newtimeroman;font-size:200%;text-align:center;border-radius: 15px 50px;">The Characteristic R Matrix For Each Digit</h3>


In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=5)
number = 0
for ax in axes.flat:
    ax.set_title(f'[{number}] Characteristic R Matrix')
    im = ax.imshow(characteristic_R_Matrix[number],cmap='jet')
    number+=1
fig.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([0.85, 0.12, 0.05, 0.7])
fig.colorbar(im, cax=cbar_ax)
plt.show()

<p style="text-align: center;"><span style='font-size: 24px; font-family: "Times New Roman", Times, serif;'>The higher values in the characteristic matrices mean that that pixel position has appeared more times as a corner of that digit. we can see that before any thresholding on our characteristic matrix that there are very high values indicating common corners among a high number of samples compared to lower values which indicate more rare samples.</span></p>

In [ ]:
for mat in characteristic_R_Matrix:
    mat[mat<mat.max()*0.6]=0


<p style="text-align: center;"><span style='font-size: 24px; font-family: "Times New Roman", Times, serif;'>In the above code cell, we apply a threshold and drop all the records of pixels under 60% of the maximum value.</span></p>
<p style="text-align: center;"><span style='font-size: 24px; font-family: "Times New Roman", Times, serif;'>This will allow us to compare new images with every characteristic matrix and check which one has the highest resemblance.</span></p>

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=5)
number = 0
for ax in axes.flat:
    ax.set_title(f'[{number}] Characteristic R Matrix')
    im = ax.imshow(characteristic_R_Matrix[number],cmap='jet')
    number+=1
fig.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([0.85, 0.12, 0.05, 0.7])
fig.colorbar(im, cax=cbar_ax)
plt.show()

<p style="text-align: center;"><span style='font-size: 24px; font-family: "Times New Roman", Times, serif;'>Above are the thresholded characteristic matrices.</span></p>

<a id="1.1"></a>
<h3 style="background-color:orange;font-family:newtimeroman;font-size:200%;text-align:center;border-radius: 15px 50px;">Prediction</h3>


<ol>
    <li><span style='font-family: "Times New Roman", Times, serif; font-size: 24px;'>To predict the value of a new image we will follow the following logic:</span></li>
    <li><span style="font-size: 24px;"><span style="font-family: 'Times New Roman', Times, serif;">&nbsp;we will compute the harris corners for the new image, the function we defined returns a matrix of zeros with the same dimensions as the digit image with 1&apos;s located at each calculated harris corner.</span></span></li>
    <li><span style="font-size: 24px;"><span style="font-family: 'Times New Roman', Times, serif;">&nbsp;we will multiply the received corner matrix from the previous stage by every characteristic matrix and sum it.</span></span><span style="font-size: 24px;"><span style="font-family: 'Times New Roman', Times, serif;"><br></span></span></li>
    <li><span style='font-family: "Times New Roman", Times, serif; font-size: 24px;'>We are left with an array of length 10 with which we will divide by the maximum value thus normalizing the values from 0 to 1. the index of the closest value to 1 is our prediction.</span></li>
</ol>

In [ ]:
def Distances_From_Characteristic_Matrixs(R_Scores):
    distances = []
    #print(R_Scores)
    for number in range(0,10):
           distances.append(np.sum((characteristic_R_Matrix[number]*R_Scores)))
    return distances/(max(distances)+0.001)

In [ ]:
Distances_From_Characteristic_Matrixs(Calculate_Harris_Corners(test_images[405]))

In [ ]:
predictions = []
for number in test_images:
    predictions.append(np.argmax(Distances_From_Characteristic_Matrixs(cv.cornerHarris(np.float32(number),HARRIS_BLOCK_SIZE,HARRIS_K_SIZE,HARIS_ALPHA))))

<a id="1.1"></a>
<h3 style="background-color:orange;font-family:newtimeroman;font-size:200%;text-align:center;border-radius: 15px 50px;">Prediction Example</h3>


In [ ]:
p_df = pd.DataFrame({'Prediction':predictions})

sample = p_df.iloc[5:11]
sample

In [ ]:
for n in range(0,6):
    plt.subplot(2,3,n+1)
    plt.title('Predicted: {}'.format(sample.iloc[n,0]))
    cor = cv.cornerHarris(np.array(test_images[sample.index[n]],dtype=np.float32),HARRIS_BLOCK_SIZE,HARRIS_K_SIZE,HARIS_ALPHA)
    cor[cor<cor.max()/1.5] = 0
    p = test_images[sample.index[n]].copy()
    p = cv.cvtColor(p.astype(np.uint8),cv.COLOR_GRAY2RGB)
    p[cor>cor.max()/3] =(255,0,0)
    plt.imshow(p)
plt.show()

<a id="1.1"></a>
<h3 style="background-color:orange;font-family:newtimeroman;font-size:200%;text-align:center;border-radius: 15px 50px;">Conclusions</h3>


<ol>
    <li style="text-align: center;"><span style='font-family: "Times New Roman", Times, serif; font-size: 24px;'>All through the current version of the algorithm as far-far away from being as accurate as even the most naive CNN, non the less we went trough the Harris corner algorithm and saw that there are key points that are more frequent in each digit than other and we defined those key points as our characteristic matrix.</span>
        <p style="text-align: center;"><span style="font-size: 24px;"><span style="font-family: 'Times New Roman', Times, serif;">There is still work to be done and maybe a more elaborate approach to the prediction evaluation can be made to increase the accuracy of the comparison.</span></span></p>
        <p style="text-align: center;"><span style="font-size: 24px;"><span style="font-family: 'Times New Roman', Times, serif;"><br></span></span></p>
        <p style="text-align: center;"><span style='font-family: "Times New Roman", Times, serif; font-size: 24px;'>Of course, I am always open to any ideas you my friends may have.</span></p>
    </li>
</ol>